# Tensorflow Input Pipeline


In [1]:
import tensorflow as tf

2025-03-14 14:13:01.878017: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-14 14:13:02.031211: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741940882.079307     373 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741940882.088221     373 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-14 14:13:02.203889: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
daily_sales_numbers = [21,22,-188,31,-1,32,34,31]

In [3]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

2025-03-14 14:13:05.758259: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [4]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-188
31
-1
32
34
31


2025-03-14 14:13:05.858662: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [5]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-188


2025-03-14 14:13:05.887032: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [6]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset:
    print(sales.numpy())

21
22
31
32
34
31


In [7]:
tf_dataset = tf_dataset.map(lambda X:X*72)
for sales in tf_dataset:
    print(sales.numpy())


1512
1584
2232
2304
2448
2232


2025-03-14 14:13:05.981378: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset:
    print(sales.numpy())

1512
2232
2304
2448
2232
1584


In [9]:
for sales in tf_dataset.batch(2):
    print(sales.numpy())

[1512 1584]
[2304 2448]
[2232 2232]


In [10]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x:x>0).map(lambda y:y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1512 2232]
[1584 2304]
[2448 2232]


## Performing on IMage Dataset

In [11]:
images_ds = tf.data.Dataset.list_files('images/*/*',shuffle=False)

for file in images_ds.take(4):
    print(file.numpy())

b'images/Cat/10000.jpg'
b'images/Cat/10001.jpg'
b'images/Cat/10002.jpg'
b'images/Cat/10003.jpg'


2025-03-14 14:13:06.654138: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
images_ds = images_ds.shuffle(200)

for file in images_ds.take(4):
    print(file.numpy())

b'images/Cat/10069.jpg'
b'images/Cat/10040.jpg'
b'images/Cat/10032.jpg'
b'images/Cat/10062.jpg'


In [13]:
class_names = ['cat','dog']

In [14]:
image_count = len(images_ds)
image_count

22561

In [15]:
train_size = int(image_count*0.5)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [16]:
len(train_ds),len(test_ds)

(11280, 11281)

In [17]:
s = 'images/Cat/10119.jpg'
s.split("/")[-2]

'Cat'

In [31]:
import os
def get_label(file_path):
    return tf.strings.split(file_path,os.path.sep)[-2]

def process_image(file_path):
    label = get_label(file_path)

    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img,[128,128])
    
    return img,label


In [32]:
for t in train_ds.take(3):
    print(t.numpy())

b'images/Cat/10105.jpg'
b'images/Cat/10071.jpg'
b'images/Cat/10017.jpg'


In [36]:
train_ds = train_ds.map(process_image)
for img, label in train_ds.take(3):
    print('Image',img)
    print("Label:",label)


Image tf.Tensor(
[[[121.78473  161.11285  129.12848 ]
  [118.51434  157.51434  129.17584 ]
  [116.83203  156.83203  130.83203 ]
  ...
  [253.02344  245.86328  230.34375 ]
  [252.65887  249.66669  237.66669 ]
  [252.17578  252.06293  240.17316 ]]

 [[127.19922  163.1836   137.21484 ]
  [121.5509   159.52728  135.54309 ]
  [118.86859  156.85297  135.89984 ]
  ...
  [252.16516  246.1914   232.54297 ]
  [254.96088  252.89069  239.39441 ]
  [252.49097  248.45972  236.42847 ]]

 [[126.86896  161.50958  137.1502  ]
  [123.640625 158.64062  137.63281 ]
  [119.03906  154.96094  137.42291 ]
  ...
  [255.       252.67969  241.25677 ]
  [251.72156  245.72156  232.4403  ]
  [253.8736   248.40094  235.11969 ]]

 ...

 [[ 60.974792  74.97479  119.97479 ]
  [ 66.76422   81.71765  126.39984 ]
  [ 65.48462   86.201294 129.04114 ]
  ...
  [251.05365  246.03516  240.03516 ]
  [247.86719  240.50781  235.58594 ]
  [245.18573  237.82635  232.90448 ]]

 [[ 68.64935   82.48401  127.48401 ]
  [ 47.101196  64.60

In [37]:
def scale(image,label):
    return image/255,label

In [39]:
train_ds = train_ds.map(scale)
for image, label in train_ds.take(4):
    print('*****Image:',image.numpy()[0][0])
    print('*****Label:',label.numpy())

*****Image: [0.00319204 0.00320742 0.00311515]
*****Label: b'Cat'
*****Image: [0.00012303 0.00012303 0.00012303]
*****Label: b'Cat'
*****Image: [0.00057454 0.00058992 0.00049765]
*****Label: b'Cat'
*****Image: [0.00173973 0.00163208 0.00138602]
*****Label: b'Cat'


2025-03-14 14:26:52.307733: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [42]:
images_ds = tf.data.Dataset.list_files('images/*/*',shuffle=False)

image_count = len(images_ds)
train_size = int(image_count*0.5)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)


tf_data = train_ds.shuffle(200).map(process_image).map(scale)
for image, label in tf_data.take(4):
    print('*****Image:',image.numpy()[0][0])
    print('*****Label:',label.numpy())

*****Image: [0.21937807 0.19584866 0.09388787]
*****Label: b'Cat'
*****Image: [0.65563726 0.42818627 0.04008023]
*****Label: b'Cat'
*****Image: [0.30980393 0.38431373 0.36078432]
*****Label: b'Cat'
*****Image: [0.31433907 0.29080966 0.2359077 ]
*****Label: b'Cat'
